# Module 5: SciPy - Comprehensive Test (SOLUTIONS)

This test covers all topics from Module 5:
- Statistics (descriptive stats, distributions, hypothesis testing)
- Interpolation and curve fitting
- Optimization (minimize, root finding)
- Integration and ODEs

**Instructions:**
1. Complete all 12 questions
2. Write your code in the provided code cells
3. Run your code to verify it works
4. Some questions have multiple parts - make sure to complete all parts

**Scoring:** Each question is worth points as indicated. Partial credit may be given.

In [ ]:
# Required imports - run this cell first
import numpy as np
from scipy import stats
from scipy import interpolate
from scipy.interpolate import interp1d, CubicSpline, UnivariateSpline
from scipy.optimize import minimize, minimize_scalar, root, brentq, curve_fit
from scipy.integrate import quad, dblquad, solve_ivp
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

print("All imports successful!")

---

## Part 1: Statistics (Questions 1-4)

### Question 1: Descriptive Statistics (8 points)

A researcher collected reaction times (in milliseconds) from 40 participants in a cognitive experiment.

Using the data provided:

1. Calculate the mean, median, and standard deviation (2 points)
2. Use `scipy.stats.describe()` to get comprehensive statistics (2 points)
3. Calculate the 25th, 50th, and 75th percentiles (IQR) (2 points)
4. Determine the skewness and interpret what it means for this data (2 points)

In [ ]:
# Reaction time data (milliseconds)
np.random.seed(42)
reaction_times = np.concatenate([
    np.random.normal(250, 30, 30),  # Normal responses
    np.random.normal(350, 40, 10)   # Slower responses (fatigue)
])

# SOLUTION

# 1. Calculate mean, median, and standard deviation
mean_rt = np.mean(reaction_times)
median_rt = np.median(reaction_times)
std_rt = np.std(reaction_times, ddof=1)  # Sample standard deviation

print("Question 1: Descriptive Statistics")
print("=" * 50)
print("\n1. Basic Statistics:")
print(f"   Mean: {mean_rt:.2f} ms")
print(f"   Median: {median_rt:.2f} ms")
print(f"   Standard Deviation: {std_rt:.2f} ms")

# 2. Use scipy.stats.describe()
description = stats.describe(reaction_times)
print("\n2. scipy.stats.describe() Results:")
print(f"   Number of observations: {description.nobs}")
print(f"   Minimum: {description.minmax[0]:.2f} ms")
print(f"   Maximum: {description.minmax[1]:.2f} ms")
print(f"   Mean: {description.mean:.2f} ms")
print(f"   Variance: {description.variance:.2f}")
print(f"   Skewness: {description.skewness:.4f}")
print(f"   Kurtosis: {description.kurtosis:.4f}")

# 3. Calculate percentiles (IQR)
q1 = np.percentile(reaction_times, 25)
q2 = np.percentile(reaction_times, 50)  # Same as median
q3 = np.percentile(reaction_times, 75)
iqr = q3 - q1

print("\n3. Percentiles:")
print(f"   25th percentile (Q1): {q1:.2f} ms")
print(f"   50th percentile (Q2/Median): {q2:.2f} ms")
print(f"   75th percentile (Q3): {q3:.2f} ms")
print(f"   Interquartile Range (IQR): {iqr:.2f} ms")

# 4. Skewness interpretation
skewness = stats.skew(reaction_times)
print("\n4. Skewness Analysis:")
print(f"   Skewness: {skewness:.4f}")
if skewness > 0.5:
    interpretation = "The data is positively (right) skewed - there's a tail of slower reaction times."
elif skewness < -0.5:
    interpretation = "The data is negatively (left) skewed."
else:
    interpretation = "The data is approximately symmetric."
print(f"   Interpretation: {interpretation}")
print(f"   This makes sense because we have some fatigued participants with slower responses.")

### Question 2: Probability Distributions (8 points)

A quality control engineer knows that the lifespan of LED bulbs follows an exponential distribution with a mean of 50,000 hours.

1. Create the exponential distribution object (2 points)
2. Calculate the probability that a bulb lasts more than 60,000 hours (2 points)
3. Calculate the probability that a bulb lasts between 30,000 and 70,000 hours (2 points)
4. Find the median lifespan (time at which 50% of bulbs fail) (2 points)

In [ ]:
# Mean lifespan
mean_lifespan = 50000  # hours

# SOLUTION

# 1. Create exponential distribution object
# For exponential distribution, scipy uses scale = mean (= 1/lambda)
exp_dist = stats.expon(scale=mean_lifespan)

print("Question 2: Exponential Distribution")
print("=" * 50)
print(f"\n1. Distribution: Exponential with mean = {mean_lifespan} hours")
print(f"   Rate parameter (lambda) = 1/{mean_lifespan} = {1/mean_lifespan:.6f}")

# 2. P(X > 60000)
prob_over_60k = 1 - exp_dist.cdf(60000)
# Alternative: exp_dist.sf(60000) gives survival function directly
print(f"\n2. P(lifespan > 60,000 hours): {prob_over_60k:.4f} ({prob_over_60k*100:.2f}%)")

# 3. P(30000 < X < 70000)
prob_30k_to_70k = exp_dist.cdf(70000) - exp_dist.cdf(30000)
print(f"\n3. P(30,000 < lifespan < 70,000 hours): {prob_30k_to_70k:.4f} ({prob_30k_to_70k*100:.2f}%)")

# 4. Median lifespan (50th percentile)
median_lifespan = exp_dist.ppf(0.5)
# Analytical: median = mean * ln(2)
median_analytical = mean_lifespan * np.log(2)
print(f"\n4. Median lifespan: {median_lifespan:.2f} hours")
print(f"   Verification (mean * ln(2)): {median_analytical:.2f} hours")
print(f"   Note: Median < Mean for exponential distribution (right-skewed)")

### Question 3: Hypothesis Testing (10 points)

A pharmaceutical company claims their new medication reduces blood pressure by an average of 15 mmHg. A clinical trial with 30 patients showed the following blood pressure reductions.

1. State the null and alternative hypotheses (2 points)
2. Perform a one-sample t-test to test the company's claim (3 points)
3. Calculate the 95% confidence interval for the mean reduction (3 points)
4. Based on the p-value (alpha = 0.05), state your conclusion (2 points)

In [ ]:
# Blood pressure reductions (mmHg) from clinical trial
np.random.seed(123)
bp_reductions = np.random.normal(loc=12.5, scale=5, size=30)

# Claimed reduction
claimed_reduction = 15  # mmHg

# SOLUTION

print("Question 3: Hypothesis Testing")
print("=" * 50)

# 1. State hypotheses
print("\n1. Hypotheses:")
print("   H0 (Null): The mean reduction equals 15 mmHg (mu = 15)")
print("   H1 (Alternative): The mean reduction does not equal 15 mmHg (mu != 15)")
print("   This is a two-tailed test.")

# Basic statistics
n = len(bp_reductions)
sample_mean = np.mean(bp_reductions)
sample_std = np.std(bp_reductions, ddof=1)
sem = stats.sem(bp_reductions)

print(f"\n   Sample size: {n}")
print(f"   Sample mean: {sample_mean:.2f} mmHg")
print(f"   Sample std: {sample_std:.2f} mmHg")

# 2. Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(bp_reductions, claimed_reduction)

print("\n2. One-Sample t-Test:")
print(f"   t-statistic: {t_stat:.4f}")
print(f"   p-value: {p_value:.4f}")

# 3. Calculate 95% confidence interval
confidence = 0.95
ci = stats.t.interval(confidence, df=n-1, loc=sample_mean, scale=sem)

print(f"\n3. 95% Confidence Interval:")
print(f"   ({ci[0]:.2f}, {ci[1]:.2f}) mmHg")
print(f"   Margin of error: +/- {(ci[1]-ci[0])/2:.2f} mmHg")

# 4. Conclusion
alpha = 0.05
print(f"\n4. Conclusion (alpha = {alpha}):")
if p_value < alpha:
    print(f"   Since p-value ({p_value:.4f}) < alpha ({alpha}), we REJECT H0.")
    print(f"   There is significant evidence that the mean reduction differs from 15 mmHg.")
else:
    print(f"   Since p-value ({p_value:.4f}) >= alpha ({alpha}), we FAIL TO REJECT H0.")
    print(f"   There is insufficient evidence to conclude the mean differs from 15 mmHg.")

# Additional interpretation
if claimed_reduction < ci[0] or claimed_reduction > ci[1]:
    print(f"\n   Note: The claimed value ({claimed_reduction} mmHg) is OUTSIDE the 95% CI.")
else:
    print(f"\n   Note: The claimed value ({claimed_reduction} mmHg) is WITHIN the 95% CI.")

### Question 4: Chi-Square Test (8 points)

A survey examined the relationship between education level and smartphone brand preference. The contingency table below shows the observed frequencies.

1. Perform a chi-square test of independence (3 points)
2. Display the expected frequencies (2 points)
3. Interpret the results at alpha = 0.05 (3 points)

In [ ]:
# Observed frequencies
# Rows: Education (High School, Bachelor's, Graduate)
# Columns: Brand (Apple, Samsung, Other)
observed = np.array([
    [30, 45, 25],   # High School
    [55, 40, 15],   # Bachelor's
    [65, 30, 15],   # Graduate
])

# SOLUTION

print("Question 4: Chi-Square Test of Independence")
print("=" * 50)

# Labels for display
education_levels = ['High School', "Bachelor's", 'Graduate']
brands = ['Apple', 'Samsung', 'Other']

# 1. Perform chi-square test
chi2, p_value, dof, expected = stats.chi2_contingency(observed)

print("\n1. Chi-Square Test Results:")
print(f"   Chi-square statistic: {chi2:.4f}")
print(f"   p-value: {p_value:.6f}")
print(f"   Degrees of freedom: {dof}")

# 2. Display observed and expected frequencies
print("\n2. Observed Frequencies:")
print(f"   {'Education':<15} {'Apple':>10} {'Samsung':>10} {'Other':>10} {'Total':>10}")
print("   " + "-" * 55)
for i, edu in enumerate(education_levels):
    print(f"   {edu:<15} {observed[i,0]:>10} {observed[i,1]:>10} {observed[i,2]:>10} {observed[i].sum():>10}")
print("   " + "-" * 55)
print(f"   {'Total':<15} {observed[:,0].sum():>10} {observed[:,1].sum():>10} {observed[:,2].sum():>10} {observed.sum():>10}")

print("\n   Expected Frequencies (if independent):")
print(f"   {'Education':<15} {'Apple':>10} {'Samsung':>10} {'Other':>10}")
print("   " + "-" * 45)
for i, edu in enumerate(education_levels):
    print(f"   {edu:<15} {expected[i,0]:>10.1f} {expected[i,1]:>10.1f} {expected[i,2]:>10.1f}")

# 3. Interpretation
alpha = 0.05
print(f"\n3. Interpretation (alpha = {alpha}):")
if p_value < alpha:
    print(f"   Since p-value ({p_value:.6f}) < alpha ({alpha}), we REJECT H0.")
    print(f"   There IS a significant relationship between education level and brand preference.")
    print(f"\n   Looking at the data patterns:")
    print(f"   - Higher education levels show stronger preference for Apple")
    print(f"   - High school graduates prefer Samsung more than other groups")
else:
    print(f"   Since p-value ({p_value:.6f}) >= alpha ({alpha}), we FAIL TO REJECT H0.")
    print(f"   There is no significant relationship between education and brand preference.")

---

## Part 2: Interpolation and Curve Fitting (Questions 5-7)

### Question 5: Interpolation (8 points)

Temperature measurements were taken at a weather station every 3 hours.

1. Create a cubic spline interpolation of the data (2 points)
2. Estimate the temperature at 10:00 AM (hour 10) and 4:30 PM (hour 16.5) (2 points)
3. Find the time and value of the maximum temperature (2 points)
4. Plot the original data points and the interpolated curve (2 points)

In [ ]:
# Time (hours from midnight) and temperature (Celsius)
hours = np.array([0, 3, 6, 9, 12, 15, 18, 21, 24])
temperatures = np.array([15, 13, 14, 19, 26, 28, 24, 19, 16])

# SOLUTION

print("Question 5: Interpolation")
print("=" * 50)

# 1. Create cubic spline
cs = CubicSpline(hours, temperatures)
print("\n1. Cubic spline created using scipy.interpolate.CubicSpline")

# 2. Estimate temperatures at specific times
temp_10am = cs(10)
temp_430pm = cs(16.5)

print(f"\n2. Temperature Estimates:")
print(f"   At 10:00 AM (hour 10): {temp_10am:.2f} C")
print(f"   At 4:30 PM (hour 16.5): {temp_430pm:.2f} C")

# 3. Find maximum temperature
# Use minimize_scalar on the negative of the spline
result = minimize_scalar(lambda x: -cs(x), bounds=(0, 24), method='bounded')
max_time = result.x
max_temp = cs(max_time)

print(f"\n3. Maximum Temperature:")
print(f"   Max temperature: {max_temp:.2f} C")
print(f"   Occurs at: {max_time:.2f} hours ({int(max_time)}:{int((max_time % 1) * 60):02d})")

# 4. Plot
hours_plot = np.linspace(0, 24, 200)
temps_plot = cs(hours_plot)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(hours, temperatures, 'ro', markersize=12, label='Measurements')
ax.plot(hours_plot, temps_plot, 'b-', linewidth=2, label='Cubic Spline')
ax.plot(10, temp_10am, 'g^', markersize=12, label=f'10:00 AM: {temp_10am:.1f}C')
ax.plot(16.5, temp_430pm, 'g^', markersize=12, label=f'4:30 PM: {temp_430pm:.1f}C')
ax.plot(max_time, max_temp, 'r*', markersize=20, label=f'Max: {max_temp:.1f}C at {max_time:.1f}h')
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Temperature (C)')
ax.set_title('Daily Temperature Interpolation')
ax.set_xticks(range(0, 25, 3))
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### Question 6: Curve Fitting (10 points)

A biologist is studying bacterial growth and collected population data over time. The data appears to follow logistic growth:

$$P(t) = \frac{K}{1 + \frac{K - P_0}{P_0} e^{-rt}}$$

where K is carrying capacity, r is growth rate, and P0 is initial population.

1. Define the logistic growth function (2 points)
2. Use `curve_fit` to fit the model to the data (3 points)
3. Extract the fitted parameters with their uncertainties (2 points)
4. Plot the data and fitted curve (3 points)

In [ ]:
# Time (hours) and bacterial population (millions)
t_data = np.array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20])
population = np.array([10, 18, 32, 55, 85, 120, 150, 175, 190, 198, 200])

# SOLUTION

print("Question 6: Curve Fitting - Logistic Growth")
print("=" * 50)

# 1. Define logistic growth function
def logistic_growth(t, K, r, P0):
    """Logistic growth model.
    
    Parameters:
        t: time
        K: carrying capacity
        r: growth rate
        P0: initial population
    """
    return K / (1 + ((K - P0) / P0) * np.exp(-r * t))

print("\n1. Logistic growth function defined:")
print("   P(t) = K / (1 + ((K - P0) / P0) * exp(-r * t))")

# 2. Fit the model using curve_fit
# Initial guesses based on data inspection
p0 = [200, 0.3, 10]  # K ~ max population, r ~ growth rate, P0 ~ initial

popt, pcov = curve_fit(logistic_growth, t_data, population, p0=p0)
K_fit, r_fit, P0_fit = popt

print("\n2. curve_fit completed successfully")

# 3. Extract parameters and uncertainties
perr = np.sqrt(np.diag(pcov))  # Standard errors
K_err, r_err, P0_err = perr

print("\n3. Fitted Parameters with Uncertainties:")
print(f"   Carrying capacity K: {K_fit:.2f} +/- {K_err:.2f} million")
print(f"   Growth rate r: {r_fit:.4f} +/- {r_err:.4f} per hour")
print(f"   Initial population P0: {P0_fit:.2f} +/- {P0_err:.2f} million")

# Calculate R-squared
y_pred = logistic_growth(t_data, *popt)
ss_res = np.sum((population - y_pred)**2)
ss_tot = np.sum((population - np.mean(population))**2)
r_squared = 1 - ss_res / ss_tot
print(f"   R-squared: {r_squared:.6f}")

# 4. Plot
t_plot = np.linspace(0, 25, 200)
p_plot = logistic_growth(t_plot, *popt)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(t_data, population, 'ro', markersize=10, label='Data')
ax.plot(t_plot, p_plot, 'b-', linewidth=2, label=f'Logistic Fit (K={K_fit:.1f}, r={r_fit:.3f})')
ax.axhline(K_fit, color='green', linestyle='--', alpha=0.5, label=f'Carrying Capacity K={K_fit:.1f}')
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Population (millions)')
ax.set_title('Bacterial Growth - Logistic Model Fit')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### Question 7: Smoothing Noisy Data (6 points)

A sensor recorded noisy measurements of a periodic signal.

1. Use `UnivariateSpline` to smooth the data with an appropriate smoothing parameter (2 points)
2. Compare at least two different smoothing levels (2 points)
3. Plot the original noisy data and smoothed curves (2 points)

In [ ]:
# Noisy sensor data
np.random.seed(42)
x_sensor = np.linspace(0, 4*np.pi, 60)
y_true = np.sin(x_sensor) + 0.3*np.sin(3*x_sensor)
y_noisy = y_true + np.random.normal(0, 0.25, len(x_sensor))

# SOLUTION

print("Question 7: Smoothing Noisy Data")
print("=" * 50)

# 1 & 2. Create splines with different smoothing levels
x_plot = np.linspace(0, 4*np.pi, 200)

# No smoothing (interpolates exactly through points)
spline_exact = UnivariateSpline(x_sensor, y_noisy, s=0)

# Light smoothing
spline_light = UnivariateSpline(x_sensor, y_noisy, s=1)

# Medium smoothing
spline_medium = UnivariateSpline(x_sensor, y_noisy, s=3)

# Heavy smoothing
spline_heavy = UnivariateSpline(x_sensor, y_noisy, s=10)

# Calculate RMSE for each
smoothing_levels = [
    (0, spline_exact, 'No Smoothing (s=0)'),
    (1, spline_light, 'Light (s=1)'),
    (3, spline_medium, 'Medium (s=3)'),
    (10, spline_heavy, 'Heavy (s=10)'),
]

print("\n1 & 2. Smoothing Comparison:")
print(f"   {'Smoothing':<20} {'RMSE vs True':>15}")
print("   " + "-" * 35)
for s, spline, name in smoothing_levels:
    y_smooth = spline(x_sensor)
    rmse = np.sqrt(np.mean((y_smooth - y_true)**2))
    print(f"   {name:<20} {rmse:>15.4f}")

# 3. Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, (s, spline, name) in enumerate(smoothing_levels):
    ax = axes[i]
    ax.plot(x_sensor, y_noisy, 'ko', alpha=0.4, markersize=4, label='Noisy Data')
    ax.plot(x_plot, spline(x_plot), 'b-', linewidth=2, label=f'Spline ({name})')
    ax.plot(x_plot, np.sin(x_plot) + 0.3*np.sin(3*x_plot), 'r--', 
            linewidth=1, alpha=0.7, label='True Signal')
    ax.set_title(name)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n   Recommendation: s=1 or s=3 provides good balance between")
print("   noise reduction and preserving the true signal shape.")

---

## Part 3: Optimization (Questions 8-10)

### Question 8: Scalar Optimization (6 points)

Find the minimum of the function:

$$f(x) = x^3 - 6x^2 + 9x + 1$$

on the interval [0, 5].

1. Use `minimize_scalar` with bounds to find the minimum (2 points)
2. Also find any local maximum in the interval (2 points)
3. Plot the function and mark the extrema (2 points)

In [ ]:
def f(x):
    return x**3 - 6*x**2 + 9*x + 1

# SOLUTION

print("Question 8: Scalar Optimization")
print("=" * 50)

# 1. Find minimum using minimize_scalar
result_min = minimize_scalar(f, bounds=(0, 5), method='bounded')

print(f"\n1. Minimum:")
print(f"   x = {result_min.x:.6f}")
print(f"   f(x) = {result_min.fun:.6f}")

# 2. Find maximum by minimizing -f(x)
result_max = minimize_scalar(lambda x: -f(x), bounds=(0, 5), method='bounded')

print(f"\n2. Maximum (found by minimizing -f):")
print(f"   x = {result_max.x:.6f}")
print(f"   f(x) = {f(result_max.x):.6f}")

# Analytical verification: f'(x) = 3x^2 - 12x + 9 = 3(x-1)(x-3) = 0
# Critical points at x = 1 (local max) and x = 3 (local min)
print(f"\n   Analytical verification:")
print(f"   f'(x) = 3x^2 - 12x + 9 = 3(x-1)(x-3)")
print(f"   Critical points: x = 1 (max), x = 3 (min)")
print(f"   f(1) = {f(1)}, f(3) = {f(3)}")

# 3. Plot
x_plot = np.linspace(0, 5, 200)
y_plot = f(x_plot)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_plot, y_plot, 'b-', linewidth=2, label=r'$f(x) = x^3 - 6x^2 + 9x + 1$')
ax.plot(result_min.x, result_min.fun, 'go', markersize=15, 
        label=f'Minimum: ({result_min.x:.2f}, {result_min.fun:.2f})')
ax.plot(result_max.x, f(result_max.x), 'r^', markersize=15, 
        label=f'Maximum: ({result_max.x:.2f}, {f(result_max.x):.2f})')
ax.axhline(0, color='gray', linestyle='--', alpha=0.3)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Finding Extrema of f(x)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### Question 9: Constrained Optimization (10 points)

A company manufactures two products. Profit is given by:

$$P(x, y) = 50x + 40y$$

Subject to constraints:
- Machine A: 2x + y <= 100 (hours)
- Machine B: x + 3y <= 90 (hours)
- x, y >= 0 (non-negative production)

1. Set up the optimization problem (maximize profit = minimize -profit) (3 points)
2. Define all constraints properly (3 points)
3. Solve and report optimal production quantities and maximum profit (4 points)

In [ ]:
# SOLUTION

print("Question 9: Constrained Optimization")
print("=" * 50)

# 1. Define objective function (minimize negative profit to maximize profit)
def neg_profit(x):
    return -(50*x[0] + 40*x[1])

print("\n1. Objective function: Minimize -P(x,y) = -(50x + 40y)")

# 2. Define constraints
# scipy.optimize uses: constraint(x) >= 0 for 'ineq' type
# Machine A: 2x + y <= 100  ->  100 - 2x - y >= 0
# Machine B: x + 3y <= 90   ->  90 - x - 3y >= 0

constraints = [
    {'type': 'ineq', 'fun': lambda x: 100 - 2*x[0] - x[1]},    # Machine A
    {'type': 'ineq', 'fun': lambda x: 90 - x[0] - 3*x[1]},     # Machine B
]

# Non-negativity constraints via bounds
bounds = [(0, None), (0, None)]  # x >= 0, y >= 0

print("\n2. Constraints:")
print("   Machine A: 2x + y <= 100  (100 - 2x - y >= 0)")
print("   Machine B: x + 3y <= 90   (90 - x - 3y >= 0)")
print("   Bounds: x >= 0, y >= 0")

# 3. Solve
x0 = [0, 0]  # Initial guess
result = minimize(neg_profit, x0, method='SLSQP', bounds=bounds, constraints=constraints)

x_opt, y_opt = result.x
max_profit = -result.fun

print(f"\n3. Optimal Solution:")
print(f"   Product X quantity: {x_opt:.2f} units")
print(f"   Product Y quantity: {y_opt:.2f} units")
print(f"   Maximum Profit: ${max_profit:.2f}")

# Verify constraints
print(f"\n   Constraint Verification:")
print(f"   Machine A hours used: {2*x_opt + y_opt:.2f} / 100")
print(f"   Machine B hours used: {x_opt + 3*y_opt:.2f} / 90")

# Visualization
fig, ax = plt.subplots(figsize=(10, 8))

# Plot constraints
x_range = np.linspace(0, 60, 200)

# Machine A: 2x + y = 100 -> y = 100 - 2x
y_A = 100 - 2*x_range
ax.plot(x_range, y_A, 'b-', linewidth=2, label='Machine A: 2x + y = 100')

# Machine B: x + 3y = 90 -> y = (90 - x)/3
y_B = (90 - x_range) / 3
ax.plot(x_range, y_B, 'r-', linewidth=2, label='Machine B: x + 3y = 90')

# Feasible region
x_fill = np.array([0, 0, 42, 50, 0])
y_fill = np.array([0, 30, 16, 0, 0])
ax.fill(x_fill, y_fill, alpha=0.3, color='green', label='Feasible Region')

# Optimal point
ax.plot(x_opt, y_opt, 'r*', markersize=20, label=f'Optimal: ({x_opt:.1f}, {y_opt:.1f})')

ax.set_xlim(0, 60)
ax.set_ylim(0, 50)
ax.set_xlabel('Product X')
ax.set_ylabel('Product Y')
ax.set_title(f'Linear Programming: Maximum Profit = ${max_profit:.2f}')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

### Question 10: Root Finding (8 points)

The Colebrook equation is used in fluid mechanics to find the friction factor f:

$$\frac{1}{\sqrt{f}} = -2 \log_{10}\left(\frac{\epsilon/D}{3.7} + \frac{2.51}{Re \sqrt{f}}\right)$$

Given: Reynolds number Re = 100000, relative roughness epsilon/D = 0.001

1. Rearrange as g(f) = 0 for root finding (2 points)
2. Use `brentq` to solve for f in the interval [0.01, 0.1] (3 points)
3. Verify your solution by substituting back (3 points)

In [ ]:
# Given parameters
Re = 100000
eps_D = 0.001  # epsilon/D

# SOLUTION

print("Question 10: Root Finding - Colebrook Equation")
print("=" * 50)

# 1. Rearrange equation: 1/sqrt(f) + 2*log10(eps_D/3.7 + 2.51/(Re*sqrt(f))) = 0
def colebrook(f, Re, eps_D):
    """Colebrook equation rearranged as g(f) = 0.
    
    Original: 1/sqrt(f) = -2*log10(eps_D/3.7 + 2.51/(Re*sqrt(f)))
    Rearranged: 1/sqrt(f) + 2*log10(eps_D/3.7 + 2.51/(Re*sqrt(f))) = 0
    """
    sqrt_f = np.sqrt(f)
    lhs = 1 / sqrt_f
    rhs = -2 * np.log10(eps_D / 3.7 + 2.51 / (Re * sqrt_f))
    return lhs - rhs

print("\n1. Colebrook equation rearranged:")
print("   g(f) = 1/sqrt(f) + 2*log10(eps_D/3.7 + 2.51/(Re*sqrt(f))) = 0")

# 2. Solve using brentq
f_solution = brentq(colebrook, 0.01, 0.1, args=(Re, eps_D))

print(f"\n2. Solution using brentq:")
print(f"   Friction factor f = {f_solution:.6f}")

# 3. Verification
sqrt_f = np.sqrt(f_solution)
lhs = 1 / sqrt_f
rhs = -2 * np.log10(eps_D / 3.7 + 2.51 / (Re * sqrt_f))

print(f"\n3. Verification:")
print(f"   LHS = 1/sqrt(f) = {lhs:.6f}")
print(f"   RHS = -2*log10(...) = {rhs:.6f}")
print(f"   |LHS - RHS| = {abs(lhs - rhs):.2e}")
print(f"   g(f) = {colebrook(f_solution, Re, eps_D):.2e} (should be ~0)")

# Visualize
f_range = np.linspace(0.01, 0.1, 200)
g_values = [colebrook(f, Re, eps_D) for f in f_range]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(f_range, g_values, 'b-', linewidth=2, label='g(f)')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.plot(f_solution, 0, 'r*', markersize=20, label=f'Root: f = {f_solution:.4f}')
ax.set_xlabel('Friction Factor f')
ax.set_ylabel('g(f)')
ax.set_title('Colebrook Equation Root Finding')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

## Part 4: Integration and ODEs (Questions 11-12)

### Question 11: Numerical Integration (8 points)

The gamma function is defined as:

$$\Gamma(n) = \int_0^\infty t^{n-1} e^{-t} dt$$

For positive integers, Gamma(n) = (n-1)!

1. Implement the gamma function using `quad` (3 points)
2. Calculate Gamma(5) and verify it equals 4! = 24 (2 points)
3. Calculate Gamma(0.5) and verify it equals sqrt(pi) (3 points)

In [ ]:
# SOLUTION

print("Question 11: Numerical Integration - Gamma Function")
print("=" * 50)

# 1. Implement gamma function using quad
def my_gamma(n):
    """Compute Gamma(n) using numerical integration.
    
    Gamma(n) = integral from 0 to infinity of t^(n-1) * e^(-t) dt
    """
    def integrand(t, n):
        return t**(n-1) * np.exp(-t)
    
    result, error = quad(integrand, 0, np.inf, args=(n,))
    return result, error

print("\n1. Gamma function implemented using quad")
print("   Gamma(n) = integral_0^inf t^(n-1) * exp(-t) dt")

# 2. Calculate Gamma(5)
gamma_5, error_5 = my_gamma(5)
analytical_5 = np.math.factorial(4)  # 4! = 24

print(f"\n2. Gamma(5):")
print(f"   Numerical result: {gamma_5:.10f}")
print(f"   Analytical (4!): {analytical_5}")
print(f"   Error estimate: {error_5:.2e}")
print(f"   Difference: {abs(gamma_5 - analytical_5):.2e}")

# 3. Calculate Gamma(0.5)
gamma_half, error_half = my_gamma(0.5)
analytical_half = np.sqrt(np.pi)

print(f"\n3. Gamma(0.5):")
print(f"   Numerical result: {gamma_half:.10f}")
print(f"   Analytical (sqrt(pi)): {analytical_half:.10f}")
print(f"   Error estimate: {error_half:.2e}")
print(f"   Difference: {abs(gamma_half - analytical_half):.2e}")

# Bonus: compare with scipy.special.gamma
from scipy.special import gamma as scipy_gamma
print(f"\n   Verification with scipy.special.gamma:")
print(f"   scipy_gamma(5) = {scipy_gamma(5):.10f}")
print(f"   scipy_gamma(0.5) = {scipy_gamma(0.5):.10f}")

### Question 12: Solving ODEs (10 points)

A pendulum's motion is described by:

$$\frac{d^2\theta}{dt^2} = -\frac{g}{L}\sin(\theta) - b\frac{d\theta}{dt}$$

where g = 9.81 m/s^2, L = 1 m (length), and b = 0.5 (damping).

Initial conditions: theta(0) = pi/4 (45 degrees), omega(0) = 0 (starts from rest)

1. Convert to a system of first-order ODEs (2 points)
2. Solve using `solve_ivp` for t = 0 to 20 seconds (3 points)
3. Plot theta vs time (2 points)
4. Create a phase portrait (theta vs omega) (3 points)

In [ ]:
# Parameters
g = 9.81  # m/s^2
L = 1.0   # m
b = 0.5   # damping coefficient

# Initial conditions
theta0 = np.pi / 4  # 45 degrees
omega0 = 0          # starts from rest

# SOLUTION

print("Question 12: Solving ODEs - Damped Pendulum")
print("=" * 50)

# 1. Convert to system of first-order ODEs
# Let y1 = theta, y2 = omega = d(theta)/dt
# Then: dy1/dt = y2 (= omega)
#       dy2/dt = -(g/L)*sin(y1) - b*y2

def pendulum(t, state, g, L, b):
    """Damped pendulum as a system of first-order ODEs.
    
    state = [theta, omega]
    d(theta)/dt = omega
    d(omega)/dt = -(g/L)*sin(theta) - b*omega
    """
    theta, omega = state
    dtheta_dt = omega
    domega_dt = -(g/L) * np.sin(theta) - b * omega
    return [dtheta_dt, domega_dt]

print("\n1. System of first-order ODEs:")
print("   d(theta)/dt = omega")
print("   d(omega)/dt = -(g/L)*sin(theta) - b*omega")

# 2. Solve using solve_ivp
t_span = (0, 20)
state0 = [theta0, omega0]

solution = solve_ivp(
    pendulum, 
    t_span, 
    state0, 
    args=(g, L, b),
    dense_output=True,
    max_step=0.05
)

print(f"\n2. solve_ivp completed:")
print(f"   Status: {solution.message}")
print(f"   Number of time steps: {len(solution.t)}")

# Evaluate solution at fine time points
t_eval = np.linspace(0, 20, 500)
sol = solution.sol(t_eval)
theta_t = sol[0]
omega_t = sol[1]

# 3 & 4. Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Theta vs time
ax1 = axes[0, 0]
ax1.plot(t_eval, np.degrees(theta_t), 'b-', linewidth=2)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Angle (degrees)')
ax1.set_title('3. Pendulum Angle vs Time')
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax1.grid(True, alpha=0.3)

# Plot 2: Angular velocity vs time
ax2 = axes[0, 1]
ax2.plot(t_eval, omega_t, 'g-', linewidth=2)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Angular Velocity (rad/s)')
ax2.set_title('Angular Velocity vs Time')
ax2.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax2.grid(True, alpha=0.3)

# Plot 3: Phase portrait
ax3 = axes[1, 0]
ax3.plot(np.degrees(theta_t), omega_t, 'b-', linewidth=1.5)
ax3.plot(np.degrees(theta0), omega0, 'go', markersize=12, label='Start')
ax3.plot(np.degrees(theta_t[-1]), omega_t[-1], 'ro', markersize=12, label='End')
ax3.set_xlabel('Angle (degrees)')
ax3.set_ylabel('Angular Velocity (rad/s)')
ax3.set_title('4. Phase Portrait (Angle vs Angular Velocity)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Energy vs time
# E = (1/2)*m*L^2*omega^2 + m*g*L*(1 - cos(theta))
# For unit mass: E = (1/2)*L^2*omega^2 + g*L*(1 - cos(theta))
KE = 0.5 * L**2 * omega_t**2
PE = g * L * (1 - np.cos(theta_t))
total_E = KE + PE

ax4 = axes[1, 1]
ax4.plot(t_eval, KE, 'b-', linewidth=1.5, label='Kinetic Energy')
ax4.plot(t_eval, PE, 'r-', linewidth=1.5, label='Potential Energy')
ax4.plot(t_eval, total_E, 'k--', linewidth=2, label='Total Energy')
ax4.set_xlabel('Time (s)')
ax4.set_ylabel('Energy (J, unit mass)')
ax4.set_title('Energy Dissipation Due to Damping')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n   Initial total energy: {total_E[0]:.4f} J")
print(f"   Final total energy: {total_E[-1]:.4f} J")
print(f"   Energy lost to damping: {total_E[0] - total_E[-1]:.4f} J ({(total_E[0] - total_E[-1])/total_E[0]*100:.1f}%)")

---

## End of Test

**Total Points: 100**

Make sure all cells have been executed and your answers are complete before submitting.